In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
# Change working directory to be current folder
import os
os.chdir('/content/gdrive/My Drive/Practice Module/Violence-Detection')

Mounted at /content/gdrive


In [2]:
import cv2
import os
from glob import glob
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

from random import shuffle

import tensorflow as tf

# Check GPU coinfiguration in Colab
print("Tensorflow version: ", tf.__version__)
print(tf.test.gpu_device_name())


Tensorflow version:  2.8.0



# Explore the dataset


In [ ]:
def check_video_number(input_path):

    file_names = glob(input_path+'/*.avi')
    fi_cnt, no_cnt = 0, 0
    for file_name in file_names:
        if file_name.split('/')[3][0:2] == 'fi':
            fi_cnt += 1
        elif file_name.split('/')[3][0:2] == 'no':
            no_cnt += 1
            
    return fi_cnt, no_cnt, (fi_cnt)/(fi_cnt+no_cnt)

In [ ]:
# only '.avi' videos are used
def load_all_file_names(input_path):

    names, labels = [], []
    random.seed(30)
   
    file_names = glob(input_path+'/*.avi')
    fi_cnt, no_cnt = 0, 0
    for file_name in file_names:
        
        if file_name.split('/')[3][0:2] == 'fi':
            fi_cnt += 1
        elif file_name.split('/')[3][0:2] == 'no':
            no_cnt += 1
    min_cnt = min(fi_cnt, no_cnt)
    
    fi_cnt, no_cnt = 0, 0    
    for file_name in file_names:    
        if file_name.split('/')[3][0:2] == 'fi':
            fi_cnt +=1
            if fi_cnt <= min_cnt:
                labels.append(1)
                names.append(file_name)
        elif file_name.split('/')[3][0:2] == 'no':
            no_cnt +=1
            if no_cnt <= min_cnt:
                labels.append(0)
                names.append(file_name)
                               
    c = list(zip(names,labels))
    shuffle(c)
    
    names, labels = zip(*c)
    return names, labels


In [ ]:
class Videoto3D:

    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

    def get_data(self, filename, length=5, skip=True):
        cap = cv2.VideoCapture(filename)
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        bAppend = False
        nframe = length * 30
        offset = int((150 - nframe)/2)
        if (nframe>=self.depth):
            if skip:
                frames = [int((x * nframe/self.depth)+offset) for x in range(self.depth)]
            else:
                frames = [x for x in range(self.depth)]
        else:
            print("Insufficient %d frames in video %s, set bAppend as True" % (nframe, filename))
            bAppend = True
            frames = [x for x in range(int(nframe))] # nframe is a float
            
        framearray = []

        for i in range(len(frames)):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])

            try:
                ret, frame = cap.read()
                frame = cv2.resize(frame, (self.height, self.width))
            except Exception as e:
                print('Exception Error', e, filename)
                continue

            framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        cap.release()
        
        if bAppend:
            while len(framearray) < self.depth:
                framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
            print("Append more frames in the framearray to have %d frames" % len(framearray))
                
        return np.array(framearray)

    
def loaddata(video_list, vid3d, length, skip=True):
    X = []
    for idx, value in enumerate(video_list):
        # Display the progress
        if (idx % 100) == 0:
            print("process data %d/%d" % (idx, len(video_list)))
        filename = value
        X.append(vid3d.get_data(filename, length, skip=skip))
        
    return np.array(X).transpose((0, 2, 3, 1))

In [ ]:
# Define parameter setting
class Args:
    batch = 128
    epoch = 50
    nclass = 2 # 11 action categories
    depth = 10
    rows = 32
    cols = 32
    skip = True # Skip: randomly extract frames; otherwise, extract first few frames

params = Args()
img_rows = params.rows
img_cols = params.cols
frames   = params.depth
channel  = 1
vid3d = Videoto3D(img_rows, img_cols, frames)
nb_classes = params.nclass


In [ ]:
#input_path = "../RWF-2000/train"
#input_path = "../opt_flow/train"
input_path = "../keypoints/train"
fight_count, no_fight_count, percent = check_video_number(input_path)
print(fight_count, no_fight_count, percent)
names, labels = load_all_file_names(input_path)
names, labels = list(names), list(labels)

# 70% training and 30% test.
train_count = int(0.8 * len(names))
test_count  = len(names) - train_count
RWF_train = names[:train_count]
RWF_test  = names[train_count:]
print(train_count, test_count)

645 463 0.5821299638989169
740 186


In [ ]:
# Prepare training data
length = 4  # length of video clip in secs
x_train = loaddata(RWF_train, vid3d, length, params.skip)
x_train = x_train.reshape((x_train.shape[0], img_rows, img_cols, frames, channel))
y_train = labels[:train_count]
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)

# Prepare test data
x_test = loaddata(RWF_test, vid3d, length, params.skip)
x_test = x_test.reshape((x_test.shape[0], img_rows, img_cols, frames, channel))
y_test = labels[train_count:]
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

# Prepare validation data
input_path = "../keypoints/val"
names_val, labels_val = load_all_file_names(input_path)
names_val, labels_val = list(names_val), list(labels_val)

x_val = loaddata(names_val, vid3d, length, params.skip)
x_val = x_val.reshape((x_val.shape[0], img_rows, img_cols, frames, channel))
y_val = labels_val
y_val = tf.keras.utils.to_categorical(y_val, nb_classes)

print("Training data", x_train.shape, y_train.shape)
print("Test data", x_test.shape, y_test.shape)
print("validation data", x_val.shape, y_val.shape)

process data 0/740
process data 100/740
process data 200/740
process data 300/740
process data 400/740
process data 500/740
process data 600/740
process data 700/740
process data 0/186
process data 100/186
process data 0/222
process data 100/222
process data 200/222
Training data (740, 32, 32, 10, 1) (740, 2)
Test data (186, 32, 32, 10, 1) (186, 2)
validation data (222, 32, 32, 10, 1) (222, 2)


In [ ]:
np.savez("datasets/c3d_keypoints_train.npz", X=x_train, Y=y_train)
np.savez("datasets/c3d_keypoints_test.npz",  X=x_test,  Y=y_test)
np.savez("datasets/c3d_keypoints_val.npz",  X=x_val,  Y=y_val)
# with np.load("c3d_train.npz") as npzfile:
#     x_train = npzfile["X"]
#     y_train = npzfile["Y"]
    
# with np.load("c3d_test.npz") as npzfile:
#     x_test = npzfile["X"]
#     y_test = npzfile["Y"]
    
# print("Training data", x_train.shape, y_train.shape)
# print("Test data", x_test.shape, y_test.shape)

In [3]:
### Get list of pre-processed datasets

In [7]:
pwd = os.getcwd()
file_list = os.listdir(os.path.join(pwd, 'datasets'))

In [8]:
file_list

['rgb_train.h5',
 'rgb_test.h5',
 'c3d_train_90f.npz',
 'c3d_test_90f.npz',
 'c3d_val_90f.npz',
 'c3d_train_120f.npz',
 'c3d_test_120f.npz',
 'c3d_val_120f.npz',
 'c3d_train_60f.npz',
 'c3d_test_60f.npz',
 'c3d_val_60f.npz',
 'c3d_val_150f.npz',
 'c3d_train_150f.npz',
 'c3d_test_150f.npz',
 'opt_flow_lstm_train.h5',
 'opt_flow_lstm_test.h5',
 'c3d_opt_flow_val.npz',
 'c3d_opt_flow_train.npz',
 'c3d_opt_flow_test.npz',
 'c3d_keypoints_train.npz',
 'c3d_keypoints_test.npz',
 'c3d_keypoints_val.npz',
 'lstm_rgb_test.h5',
 'lstm_rgb_train.h5',
 'lstm_rgb_val.h5',
 'lstm_opt_flow_val.h5',
 'lstm_opt_flow_train.h5',
 'lstm_opt_flow_test.h5',
 'lstm_keypoints_train.h5',
 'lstm_keypoints_test.h5',
 'lstm_keypoints_val.h5']

In [12]:
os.listdir(pwd+'/../RWF-2000')

['train', 'val']

In [13]:
os.listdir(pwd+'/../opt_flow')

['train', 'val']

In [14]:
os.listdir(pwd+'/../keypoints')

['train', 'val']